# Competition

In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [20]:
class Config:
    seed = 1234
    data_dir = '../data/'
    
    feature_dict_book = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    feature_dict_trade = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    model_params = {
        "xgb_bl": {
            "objective": "reg:squarederror",
            "booster": "gbtree",
            "nthread": -1,
            "eta": 0.3,
            "max_depth": 8,
            "min_child_weight": 1,
            "sampling_method": "gradient_based",
#             "tree_method": "gpu_hist"  # turn it on for GPU
        }
    }

In [21]:
config = Config

## Preprocessing

### Preprocessing Utilities

In [4]:
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [5]:
def log_return(wap):
    return np.log(wap).diff()

In [6]:
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [7]:
def count_unique(series):
    return len(np.unique(series))

In [19]:
# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv(config.data_dir + 'train.csv')
    test = pd.read_csv(config.data_dir + 'test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

### Preprocessing of Book

In [8]:
from functools import reduce

def book_preprocessor(file_path):
    # Function to preprocess book data (for each stock id)
    
    df = pd.read_parquet(file_path)
    
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        "bid_ask_spread":[np.sum, np.mean, np.std],
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Function to get group stats for different windows (seconds in bucket)
        
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
            df_feature.rename({f'time_id__{seconds_in_bucket}': 'time_id_'}, axis=1, inplace=True)
        return df_feature
    
    # Get the stats for different windows
    df_list = [get_stats_window(seconds_in_bucket = 0, add_suffix = False)]
    
    time_gap = 100
    time_slices = [t * 100 for t in range(600 // time_gap)]
    for t in time_slices:
        df_list += [get_stats_window(seconds_in_bucket = t, add_suffix = True)]
    
    df_feature = reduce(lambda left, right: pd.merge(left, right, on='time_id_'), df_list)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature.loc[:, 'row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    
    return df_feature

### Preprocessing of Trade

In [9]:
def trade_preprocessor(file_path):
    # Function to preprocess trade data (for each stock id)
    
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count':[np.mean,np.sum,np.max],
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Function to get group stats for different windows (seconds in bucket)
        
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
            df_feature.rename({f'time_id__{seconds_in_bucket}': 'time_id_'}, axis=1, inplace=True)
        return df_feature
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    def process_trade_features(df):
        lis = []
        for n_time_id in df['time_id'].unique():
            df_id = df[df['time_id'] == n_time_id]        
            tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
            f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
            f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
            df_max =  np.sum(np.diff(df_id['price'].values) > 0)
            df_min =  np.sum(np.diff(df_id['price'].values) < 0)
            abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
            energy = np.mean(df_id['price'].values**2)
            iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
            abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
            energy_v = np.sum(df_id['size'].values**2)
            iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)

            lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                       'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})

        df_lr = pd.DataFrame(lis)
        return df_lr
    
    # Get the stats for different windows
    df_list = [get_stats_window(seconds_in_bucket = 0, add_suffix = False)]
    
    time_gap = 100
    time_slices = [t * 100 for t in range(600 // time_gap)]
    for t in time_slices:
        df_list += [get_stats_window(seconds_in_bucket = t, add_suffix = True)]
        
    df_feature = reduce(lambda left, right: pd.merge(left, right, on='time_id_'), df_list)
    df_lr = process_trade_features(df)
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    
    return df_feature

### Preprocess of Time Stock

In [10]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = [col for col in df.columns if 'realized_volatility' in col]

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

### Overall Preprocessing

In [12]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = config.data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = config.data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = config.data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = config.data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

In [13]:
df = preprocessor([0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   30.9s finished


In [14]:
df

,wap1_sum,wap1_mean,wap1_std,wap2_sum,wap2_mean,wap2_std,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,wap_balance_sum,wap_balance_mean,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_std,price_spread2_sum,price_spread2_mean,price_spread2_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,bid_ask_spread_sum,bid_ask_spread_mean,bid_ask_spread_std,wap1_sum_0,wap1_mean_0,wap1_std_0,wap2_sum_0,wap2_mean_0,wap2_std_0,log_return1_sum_0,log_return1_realized_volatility_0,log_return1_mean_0,log_return1_std_0,log_return2_sum_0,log_return2_realized_volatility_0,log_return2_mean_0,log_return2_std_0,wap_balance_sum_0,wap_balance_mean_0,wap_balance_std_0,price_spread_sum_0,price_spread_mean_0,price_spread_std_0,price_spread2_sum_0,price_spread2_mean_0,price_spread2_std_0,bid_spread_sum_0,bid_spread_mean_0,bid_spread_std_0,ask_spread_sum_0,ask_spread_mean_0,ask_spread_std_0,total_volume_sum_0,total_volume_mean_0,total_volume_std_0,volume_imbalance_sum_0,volume_imbalance_mean_0,volume_imbalance_std_0,bid_ask_spread_sum_0,bid_ask_spread_mean_0,bid_ask_spread_std_0,wap1_sum_100,wap1_mean_100,wap1_std_100,wap2_sum_100,wap2_mean_100,wap2_std_100,log_return1_sum_100,log_return1_realized_volatility_100,log_return1_mean_100,log_return1_std_100,log_return2_sum_100,log_return2_realized_volatility_100,log_return2_mean_100,log_return2_std_100,wap_balance_sum_100,wap_balance_mean_100,wap_balance_std_100,price_spread_sum_100,price_spread_mean_100,price_spread_std_100,price_spread2_sum_100,price_spread2_mean_100,price_spread2_std_100,bid_spread_sum_100,bid_spread_mean_100,bid_spread_std_100,ask_spread_sum_100,ask_spread_mean_100,ask_spread_std_100,total_volume_sum_100,total_volume_mean_100,total_volume_std_100,volume_imbalance_sum_100,volume_imbalance_mean_100,volume_imbalance_std_100,bid_ask_spread_sum_100,bid_ask_spread_mean_100,bid_ask_spread_std_100,wap1_sum_200,wap1_mean_200,wap1_std_200,wap2_sum_200,wap2_mean_200,wap2_std_200,log_return1_sum_200,log_return1_realized_volatility_200,log_return1_mean_200,log_return1_std_200,log_return2_sum_200,log_return2_realized_volatility_200,log_return2_mean_200,log_return2_std_200,wap_balance_sum_200,wap_balance_mean_200,wap_balance_std_200,price_spread_sum_200,price_spread_mean_200,price_spread_std_200,price_spread2_sum_200,price_spread2_mean_200,price_spread2_std_200,bid_spread_sum_200,bid_spread_mean_200,bid_spread_std_200,ask_spread_sum_200,ask_spread_mean_200,ask_spread_std_200,total_volume_sum_200,total_volume_mean_200,total_volume_std_200,volume_imbalance_sum_200,volume_imbalance_mean_200,volume_imbalance_std_200,bid_ask_spread_sum_200,...,wap_balance_std_400,price_spread_sum_400,price_spread_mean_400,price_spread_std_400,price_spread2_sum_400,price_spread2_mean_400,price_spread2_std_400,bid_spread_sum_400,bid_spread_mean_400,bid_spread_std_400,ask_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,total_volume_sum_400,total_volume_mean_400,total_volume_std_400,volume_imbalance_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,bid_ask_spread_sum_400,bid_ask_spread_mean_400,bid_ask_spread_std_400,wap1_sum_500,wap1_mean_500,wap1_std_500,wap2_sum_500,wap2_mean_500,wap2_std_500,log_return1_sum_500,log_return1_realized_volatility_500,log_return1_mean_500,log_return1_std_500,log_return2_sum_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,wap_balance_sum_500,wap_balance_mean_500,wap_balance_std_500,price_spread_sum_500,price_spread_mean_500,price_spread_std_500,price_spread2_sum_500,price_spread2_mean_500,price_spread2_std_500,bid_spread_sum_500,bid_spread_mean_500,bid_spread_std_500,ask_spread_sum_500,ask_spread_mean_500,ask_spread_std_500,total_volume_sum_500,total_volume_mean_500,

/Users/hlu/anaconda3/envs/volatility/lib/python3.8/site-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


## Traning

### Utilities

In [31]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, xgb_train):
    y_true = xgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred)

### Modeling

In [32]:
def train_and_evaluate(train, test):
    # Hyperparammeters (just basic)
    params = {
        "objective": "reg:squarederror",
        "booster": "gbtree",
        "nthread": -1,
        "eta": 0.3,
        "max_depth": 8,
        "min_child_weight": 1,
        "sampling_method": "gradient_based",
#         "tree_method": "gpu_hist"  # turn it on for GPU
    }
    
    # Split features and target
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    x_test = test.drop(['row_id', 'time_id'], axis = 1)
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    x_test['stock_id'] = x_test['stock_id'].astype(int)
    dtest = xgb.DMatrix(x_test)
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = config.seed, shuffle = True)
    # Iterate through each fold
    for fold, (train_idx, val_idx) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
        x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

        x_train["stock_id"] = x_train["stock_id"].astype(int)
        x_val["stock_id"] = x_val["stock_id"].astype(int)

        dtrain = xgb.DMatrix(x_train, label=y_train, weight=1/np.square(y_train), enable_categorical=True)
        dval = xgb.DMatrix(x_val, label=y_val, weight=1/np.square(y_val), enable_categorical=True)

        model = xgb.train(params,
                          dtrain=dtrain,
                          num_boost_round=100,
                          evals=[(dtrain, "dtrain"), (dval, "dval")],
                          verbose_eval=10,
                          feval=feval_rmspe)

        oof_predictions[val_idx] = model.predict(dval)
        test_predictions += model.predict(dtest) / 5
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

In [ ]:
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

### Running

In [23]:
# Read train and test
train, test = read_train_test()

# Get unique stock ids 
train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   30.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.8s finished


In [33]:
# Traing and evaluate
test_predictions = train_and_evaluate(train, test)
# Save test predictions
test['target'] = test_predictions
test[['row_id', 'target']].to_csv('submission.csv',index = False)

Training fold 1
[0]	dtrain-rmse:0.34874	dtrain-RMSPE:161.45158	dval-rmse:0.34874	dval-RMSPE:161.20300
[10]	dtrain-rmse:0.00988	dtrain-RMSPE:4.57245	dval-rmse:0.00989	dval-RMSPE:4.57050
[20]	dtrain-rmse:0.00069	dtrain-RMSPE:0.32170	dval-rmse:0.00073	dval-RMSPE:0.33865
[30]	dtrain-rmse:0.00060	dtrain-RMSPE:0.27676	dval-rmse:0.00064	dval-RMSPE:0.29700
[40]	dtrain-rmse:0.00057	dtrain-RMSPE:0.26397	dval-rmse:0.00062	dval-RMSPE:0.28652
[50]	dtrain-rmse:0.00055	dtrain-RMSPE:0.25597	dval-rmse:0.00061	dval-RMSPE:0.28077
[60]	dtrain-rmse:0.00054	dtrain-RMSPE:0.24883	dval-rmse:0.00060	dval-RMSPE:0.27531
[70]	dtrain-rmse:0.00053	dtrain-RMSPE:0.24414	dval-rmse:0.00059	dval-RMSPE:0.27218
[80]	dtrain-rmse:0.00052	dtrain-RMSPE:0.24038	dval-rmse:0.00058	dval-RMSPE:0.26984
[90]	dtrain-rmse:0.00051	dtrain-RMSPE:0.23700	dval-rmse:0.00058	dval-RMSPE:0.26851
[99]	dtrain-rmse:0.00051	dtrain-RMSPE:0.23450	dval-rmse:0.00058	dval-RMSPE:0.26739
Training fold 2
[0]	dtrain-rmse:0.34874	dtrain-RMSPE:161.40588	dval-

KeyboardInterrupt: 